In [ ]:
class Network:
    def __init__(self):
        pass
    
    
    
    def setup_network(self,number_of_user_pairs,distance_between_users,number_of_time_slots,
                      number_of_storages,spike_mean,num_spikes,storage_capacity,
                      fidelity_threshold_range,storage_node_selection_scheme,
                      num_paths,number_of_storages
                     ):
        self.reset_variables()
        self.get_user_pairs(number_of_user_pairs,distance_between_users,number_of_time_slots)
        #work_load = Work_load(number_of_time_slots,"time_demands_file.csv")
        """we set the demands for each user pair"""
        if setting_demands=="python_library":
            self.set_each_user_pair_demands(number_of_time_slots,network.each_t_user_pairs,spike_mean,num_spikes)
        else:
            self.set_each_user_pair_demands_randomly(number_of_time_slots,network.each_t_user_pairs,spike_mean,num_spikes)
        """we set at least one demand for each time to avoid divided by zero error"""
        self.check_demands_per_each_time(network.each_t_user_pairs)
        for storage_capacity in storage_capacities:
            for fidelity_threshold_range in fidelity_threshold_ranges:
                self.fidelity_threshold_range = fidelity_threshold_range
                self.set_each_request_fidelity_threshold()
#                             print("self.each_request_threshold ",network.each_request_threshold)
                for storage_node_selection_scheme in storage_node_selection_schemes:
                    selected_storage_nodes = []
                    selected_storage_pairs = []
                    for num_paths in [1]:
                        self.reset_storage_pairs()
                        for number_of_storages in [0,2,4,8,10]:
                            try:
                                """with new storage pairs, we will check the solution for each number of paths(real and virtual)"""

                                pairs = []
                                """select and add new storage pairs"""
                                available_flag = network.get_new_storage_pairs(number_of_storages,storage_node_selection_scheme)
                                self.set_each_storage_fidelity_threshold()
                                self.set_paths_in_the_network()
